In [3]:
import mne 
import numpy as np
import os
import pyxdf

In [77]:
raw_data_path = r'D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg.xdf'
streams, header = pyxdf.load_xdf(raw_data_path)


In [78]:
for i, s in enumerate(streams):
    print(f"Stream {i}: {s['info']['name'][0]}, {s['info']['type'][0]}, shape={s['time_series'].shape}")

Stream 0: PsychoPy_LSL, Markers, shape=(1598, 1)
Stream 1: Pilot02_cogwear, EEG, shape=(162660, 8)


In [79]:
eeg_stream = streams[1]  # Pilot02_cogwear
marker_stream = streams[0]  # PsychoPy_LSL

# --- 3. Prepare EEG data
data = eeg_stream['time_series'].T  # shape (n_channels, n_samples)
sfreq = float(eeg_stream['info']['nominal_srate'][0])
# --- Try to get channel names ---
desc = eeg_stream['info']['desc'][0] if eeg_stream['info']['desc'] else None

if desc and 'channels' in desc[0] and 'channel' in desc[0]['channels'][0]:
    ch_names = [ch['label'][0] for ch in desc[0]['channels'][0]['channel']]
else:
    # If no channel labels in file, create generic ones
    n_channels = data.shape[0]
    ch_names = [f'EEG{i+1}' for i in range(n_channels)]
print(f"Channel names: {ch_names}")

Channel names: ['EEG1', 'EEG2', 'EEG3', 'EEG4', 'EEG5', 'EEG6', 'EEG7', 'EEG8']


In [80]:
# --- Create MNE Raw object ---
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
raw = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=8, n_times=162660
    Range : 0 ... 162659 =      0.000 ...   650.636 secs
Ready.


In [81]:
# Marker data
marker_times = marker_stream['time_stamps']
marker_values = [m[0] for m in marker_stream['time_series']]

# Align marker timestamps to EEG
start_time = eeg_stream['time_stamps'][0]
event_samples = ((marker_times - start_time) * sfreq).astype(int)

# Keep original trigger values
events = np.array([[s, 0, val] for s, val in zip(event_samples, marker_values)])

print("First few events:", events[:10])



First few events: [[ 9886     0    71]
 [ 9887     0    31]
 [ 9994     0     3]
 [10285     0    90]
 [10381     0    74]
 [10382     0    23]
 [10490     0    16]
 [10844     0    90]
 [10987     0    73]
 [10988     0    26]]


In [82]:
n_times = raw.n_times
stim_data = np.zeros((1, n_times))

# Fill trigger channel with event codes
for sample, _, event_id in events:
    if sample < n_times:
        stim_data[0, sample] = event_id

# Create and add stim channel
stim_info = mne.create_info(['STI 014'], sfreq=raw.info['sfreq'], ch_types=['stim'])
stim_raw = mne.io.RawArray(stim_data, stim_info)

raw.add_channels([stim_raw])

# Map your active channels to the montage names
raw.rename_channels({
    'EEG1': 'AF8',
    'EEG2': 'Fp2',
    'EEG3': 'Fp1',
    'EEG4': 'AF7'
})

raw.drop_channels(['EEG5','EEG6','EEG7','EEG8'])

# Now save
raw.save(raw_data_path.replace('.xdf', '-raw.fif'), overwrite=True)

Creating RawArray with float64 data, n_channels=1, n_times=162660
    Range : 0 ... 162659 =      0.000 ...   650.636 secs
Ready.
Overwriting existing file.
Writing D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif
Overwriting existing file.
Closing D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif
[done]


[WindowsPath('D:/Priming_name/Pilot02_cogwear/sub-Pilot02/ses-S001/eeg/sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif')]

In [83]:
filt_raw = raw.filter(l_freq=1., h_freq=40.)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 s)



In [84]:
montage_dict = {
    'AF8': [0.35, 0.12, 0.0],  # Right front
    'Fp2': [0.2, 0.1, 0.0],    # Right frontal pole
    'Fp1': [-0.2, 0.1, 0.0],   # Left frontal pole
    'AF7': [-0.35, 0.12, 0.0], # Left front
}

# Create the montage
montage = mne.channels.make_dig_montage(ch_pos=montage_dict, coord_frame='head')

# Assign the montage to your Raw object
filt_raw.set_montage(montage, match_case=False, on_missing='ignore')

<RawArray | 5 x 162660 (650.6 s), ~6.2 MiB, data loaded>

In [85]:
print(filt_raw.ch_names)


['AF8', 'Fp2', 'Fp1', 'AF7', 'STI 014']


In [61]:
events = mne.read_events(r'D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw_face_onset_copy-events.fif')


C:\Users\LSA\AppData\Local\Temp\ipykernel_23196\1680504022.py:1: RuntimeWarning: This filename (D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw_face_onset_copy-events.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(r'D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw_face_onset_copy-events.fif')


In [64]:
tmin = -0.2  # 200 ms before event
tmax = 1.0   # 800 ms after event
baseline = (-0.2, 0)

epochs = mne.Epochs(filt_raw, events, 171, tmin, tmax,
                    baseline=baseline, preload=True)

Not setting metadata
100 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 301 original time points ...
0 bad epochs dropped


In [65]:
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs
ica = ICA(n_components=4,method='picard', max_iter="auto", random_state=97)
# filt_raw.drop_channels(["EXG1", "EXG2"])
# filt_raw.drop_channels(['EEG5', 'EEG6', 'EEG7', 'EEG8'])
ica.fit(epochs)

Fitting ICA to data using 4 channels (please be patient, this may take a while)
Selecting by number: 4 components
Fitting ICA took 0.1s.


C:\Users\LSA\AppData\Local\Temp\ipykernel_23196\468961516.py:5: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Method,picard
Fit parameters,max_iter=500
Fit,11 iterations on epochs (30100 samples)
ICA components,4
Available PCA components,4
Channel types,eeg
ICA components marked for exclusion,—


In [66]:
ica.plot_sources(epochs, show_scrollbars=True)  # plot component signals
# ica.plot_properties(filt_raw, picks=[0, 1, 2, 3])  # detailed plots for components


Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


In [67]:
folder = os.path.dirname(raw_data_path)
basename = os.path.splitext(os.path.basename(raw_data_path))[0]

ica_path = os.path.join(folder, f'{basename}-ica.fif')
ica.save(ica_path, overwrite=True)

Writing ICA solution to D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-ica.fif...


Method,picard
Fit parameters,max_iter=500
Fit,11 iterations on epochs (30100 samples)
ICA components,4
Available PCA components,4
Channel types,eeg
ICA components marked for exclusion,ICA001ICA002
